### Python Study Group week 4

### Theme: 使用 Selenium 爬取 Image 

------------------------------------------------------------

### Let's get it started !

<br>

### Review : Selenium
```
Selenium 原為網頁測試工具，但由於它可以直接以程式碼操控瀏覽的特性，使其成為網頁爬蟲的重要工具之一。
下面會介紹如何使用 selenium 呼叫 Chrome 瀏覽器。

Webdriver 支援 Chrome, Firefox 等瀏覽器。
```

In [14]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm_notebook

import numpy as np
import pandas as pd
import os, sys
import urllib

### 使用 Selenium 之 Webdriver 開啟 PTT 表特版
```
以下，示範如何透過幾行程式碼呈現了三個步驟：
1. webdriver.Chrome('....') -> 載入 Chrome driver。
2. driver.get('https:/....') -> 瀏覽 ptt 表特版首頁。
3. driver.find_element_by_class_name('over18-button-container').click() -> 點選已滿 18 歲按鈕。
4. driver.close() -> 關閉瀏覽頁面。

Note:
因 ptt 表特版必須年滿 18 歲才能造訪，故必須點選按鈕後再進行網頁解析。

若使用先前所說的 requests 模組，就可直接在 requests 的 HTTP method 中動手腳即可：
-> request.get(
    url=url,
    cookies={'over18':'1'}
   )
```

In [19]:
# Example:
driver = webdriver.Chrome('/Users/samhsia/政大/Python_Study_Group/chromedriver')

pttbeauty_url = 'https://www.ptt.cc/bbs/Beauty/index.html'
driver.get(pttbeauty_url)
driver.find_element_by_class_name('over18-button-container').click()
driver.close()

### 撰寫對網站發出請求(requests)的function
```
通過篩選條件的發文，以使 function 發送請求。
```

In [16]:
def get_web_page(url):
    resp = requests.get(
        url = url,
        cookies={'over18': '1'}
    )
    if resp.status_code != 200:
        print('Invalid url: ', resp.url)
        return None
    else:
        return resp.text

### 解析頁面爬取通過篩選條件的圖片，並儲存到資料夾中。
```
在點選通過 18 歲的按鈕後，我們進到表特版的首頁。
接著就能夠過 BeautifulSoup 去解析，並爬取圖片。

Note:
    創建資料夾：os.mkdir()
```

In [17]:
if __name__ == '__main__':
    # Open the webdriver
    driver = webdriver.Chrome('/Users/samhsia/政大/Python_Study_Group/chromedriver')
    pttbeauty_url = 'https://www.ptt.cc/bbs/Beauty/index.html'
    
    # get/dive into web page
    driver.get(pttbeauty_url)
    driver.find_element_by_class_name('over18-button-container').click()
    
    # make directory
    if not os.path.exists('image_folder'):
        os.mkdir('image_folder')
    else:
        pass

    # Parse the page
    soup = BeautifulSoup(driver.page_source)
    main_container = soup.find('div', {'id': 'main-container'})
    divs = main_container.find_all('div', class_='r-ent')

    for div in tqdm_notebook(divs):
        push_count = 0
        push_str = div.find('div', class_='nrec').text
        title = div.find('div', class_='title').text.strip()

        if push_str:
            try:
                push_count = int(push_str)
            except ValueError:
                if push_str == '爆':
                    push_count = 99
                elif push_str.startswith('X'):
                    push_count = -10  

        # filter with condition & get url
        if not title.startswith('[公告]') and push_count > 5:
            beauty_url = "{0}{1}".format('https://www.ptt.cc', div.find('a')['href'])

            resp = get_web_page(beauty_url)
            beauty_soup = BeautifulSoup(resp, 'html.parser')
            img_divs = beauty_soup.find_all('div', class_='richcontent')

            # set the file route
            file_path = '/Users/samhsia/政大/Python_Study_Group/image_folder'
            full_filename = os.path.join(file_path, title)

            # store the image to the folder with corresponding filenames
            for i, d in enumerate(img_divs):
                url = d.find('a')['href']
                url = "https:" + url + '.jpg'
                urllib.request.urlretrieve(url, "{0}{1}{2}_.jpg".format(full_filename, '_', i)) 

    driver.close()